# COGS 108 - EDA Checkpoint

# Names

- Mateo Ignacio
- Samuel Piltch
- Nate del Rosario 🐐
- Lisa Hwang
- Geovaunii D. White

<a id='research_question'></a>
# Research Question

How do the audio features from songs, specifically Spotify Tracks compare to each other? Is there a relationship between some of these features such as tempo correlating with danceability/energy/liveness and if so how are they correlated. Additionally, how can we use these features to cluster songs based on these audio tracks of songs being coverted to numeric features?

# Setup

In [16]:
pip install plotly==5.14.1
pip install -U scikit-learn

SyntaxError: invalid syntax (3576522951.py, line 1)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_selection import SelectKBest, chi2

ModuleNotFoundError: No module named 'plotly'

# Data Cleaning

Describe your data cleaning steps here.

In [ ]:
# define our features
features_array = np.array(charts.columns)
features = np.delete(features_array, [0, 1])
features

NameError: name 'np' is not defined

In [ ]:
# convert milliseconds to minutes
charts = charts.assign(duration_min = charts['duration'] / 60000).drop(columns=['duration'])

# feature scaling
scaler = MinMaxScaler()
scaler.fit(np.array(charts[['tempo', 'loudness', 'duration_min', 'key', 'mode']]))
charts[['tempo', 'loudness', 'duration_min', 'key', 'mode']] = scaler.transform(np.array(charts[['tempo', 'loudness', 'duration_min', 'key', 'mode']]))

# Scaled!
charts

In [ ]:
for col in charts.columns:
    num_NaN = charts[col].isna().sum()
    print("Are there any NaN's?")
    if num_NaN == 0:
        print('No')

# Data Analysis & Results (EDA)

Carry out EDA on your dataset(s); Describe in this section

In [ ]:
charts.describe()